In [3]:
!pip install chromadb

import pandas
from datasets import Dataset
from transformers import T5Tokenizer,T5ForConditionalGeneration,TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import chromadb
from chromadb.utils import embedding_functions
import ast

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-win_amd64.whl.metadata (262 bytes)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached onnxruntime-1.20.1-cp310-cp310-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.30.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.51b0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached kubernetes-32.0.1-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached mmh3-5.1.0-cp310-cp310-win_amd64.whl.metadata (16 kB)
  Using cached orjson-3.10.15-cp310-


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
data=pandas.read_csv('qa_pairs.csv')

In [5]:
qa_pairs = pandas.DataFrame(data)

In [6]:
qa_pairs['question'].sample(6)

10112    Who may derive many benefits from worshipping ...
11048    What should a king do even when flushed with v...
5871     What type of yoga yields success in all undert...
11599    What is the purpose of worshipping on the eigh...
12464                    How many sons did the woman lose?
9122     How should a seeker of salvation meditate upon...
Name: question, dtype: object

In [7]:
def process_data(qa_pairs):
    data=[]
    for i,row in qa_pairs.iterrows():
        question = row['question']
        answer = row['answer']
        context = row['verse_text']

        input_text = f'question : {question} context : {context}'
        output = answer

        data.append({f"input_text" : input_text, 'target_text' : output })
    return data
    
training_data = process_data(qa_pairs)


# 
train_hf = Dataset.from_list(training_data)

train_data, eval_data = train_test_split(training_data, test_size=0.2)

train_df = pandas.DataFrame(train_data)
eval_df = pandas.DataFrame(eval_data)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

model_name='t5-small'
tokenizer= T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["target_text"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# ✅ Step 6: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/19728 [00:00<?, ? examples/s]

Map:   0%|          | 0/4932 [00:00<?, ? examples/s]

In [8]:
# ✅ Ensure model is in training mode
model.train()

# ✅ Ensure all parameters require gradients
for param in model.parameters():
    param.requires_grad = True  

# ✅ Fix potential gradient checkpointing issue
model.config.use_cache = False  

for name, param in model.named_parameters():
    if param.requires_grad and param.grad is None:
        print(f"⚠️ No gradient for {name} (Before Training)")

⚠️ No gradient for shared.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.0.SelfAttention.q.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.0.SelfAttention.k.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.0.SelfAttention.v.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.0.SelfAttention.o.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.0.layer_norm.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.1.DenseReluDense.wi.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.1.DenseReluDense.wo.weight (Before Training)
⚠️ No gradient for encoder.block.0.layer.1.layer_norm.weight (Before Training)
⚠️ No gradient for encoder.block.1.layer.0.SelfAttention.q.weight (Before Training)
⚠️ No gradient for encoder.block.1.layer.0.SelfAttention.k.weight (Before Training)
⚠️ No gra

In [ ]:
# ✅ Step 7: Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

trainer.train()

# ✅ Step 8: Save the Fine-Tuned Model
model.save_pretrained("t5_karma_finetuned")
tokenizer.save_pretrained("t5_karma_finetuned")

print("🎉 Training Complete! Model saved as 't5_karma_finetuned'.")

In [9]:
data=pandas.read_csv('Content_Storage_df.csv')
embedding_df=pandas.DataFrame(data)
embedding_df.head(1)
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_collection("karma_embeddings")

BATCH_SIZE = 166  # ChromaDB's max batch size

def insert_data(df):
    # Ensure embeddings are in the correct format (list of floats)
    df["embedding_vector"] = df["embedding_vector"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Convert verse_id to float format for uniqueness
    df["verse_id"] = df.apply(lambda row: float(f"{row['chapter_id']}.{row['verse_id']}"), axis=1)

    # Insert data in batches
    for i in range(0, len(df), BATCH_SIZE):
        batch = df.iloc[i : i + BATCH_SIZE]

        collection.add(
            ids=batch["verse_id"].astype(str).tolist(),      # Convert verse_id to string
            documents=batch["verse_text"].tolist(),          # List of verses
            embeddings=batch["embedding_vector"].tolist()    # List of embeddings
        )
        print(f"Inserted {len(batch)} records...")

    print("✅ All data inserted successfully!")

# Call function to insert your data
insert_data(embedding_df)

Inserted 166 records...
Inserted 166 records...
Inserted 166 records...
Inserted 166 records...
Inserted 166 records...
Inserted 166 records...
Inserted 166 records...
Inserted 73 records...
✅ All data inserted successfully!


In [28]:
model = T5ForConditionalGeneration.from_pretrained('t5_karma_finetuned')
tokenizer = T5Tokenizer.from_pretrained('t5_karma_finetuned')


def retrieve_context(question, top_k=3):
    embedding_fn = embedding_functions.DefaultEmbeddingFunction()
    question_embedding = embedding_fn([question])[0]

    # Retrieve top-k matching documents
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )
    if results["documents"]:
        # print(results["documents"])
        flat_documents = [doc for sublist in results["documents"] for doc in sublist]
        return " ".join(flat_documents) if flat_documents else "No relevant context found."
    
    return "No relevant context found."

def get_answer(question, context):
    input_text = f"question: {question}  context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    output_ids = model.generate(input_ids, max_length=128)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return answer



'he is the destroyer of kartavirya'

In [6]:
while True:
    question = input('user : ')
    context = retrieve_context(question)
    answer = get_answer(question,context)
    final_answer ='user : '+ answer

user :  is sexually desired women attracted to other men?


NameError: name 'retrieve_context' is not defined